In [ ]:
import csv
import pandas as pd
from pprint import pprint 
from pathlib import Path

In [ ]:
samples = []

is_clean = False
target_file = "batch_results_4000.csv"

clean_csv_file = []
same_tag_list = []
diff_tag_list = []
with open(target_file, 'r') as f:
    df = pd.read_csv(f)
    clean_df = pd.DataFrame()
    
    if is_clean:
        clean_df = df
        if 'labels' not in clean_df.keys():
            clean_df = clean_df.rename(columns={"effectiveness" : "labels"})
    
    else:
        for key in df.keys():
            if "Input." in key:
                new_key = key.split('Input.')[-1]
                clean_df[new_key] = df[key]
            
            if "Answer" in key:
                clean_df["labels"] = df[key] 

    for idx in range(len(clean_df)):
        line = clean_df.iloc[idx]
        
        line_dict = line.to_dict()
        line_dict['labels'] = 'O' if line_dict['labels']=='[{"effectiveness":"yes"}]' or line_dict['labels'] == 'O' else 'X'
        
        found = False
        for sample in samples:
            if sample['q_id'] == line_dict['q_id'] and sample['intermediate_question'] == line_dict['intermediate_question']:
                found=True
                if sample['labels'] == line_dict['labels']:
                    same_tag_list.append(line_dict)
                else:
                    diff_tag_list.append(sample)
                    diff_tag_list.append(line_dict)
        if not found:
            samples.append(line_dict)
            
print(len(clean_df))
print(len(same_tag_list))
print(len(diff_tag_list) / 2)

assert len(same_tag_list) * 2 + len(diff_tag_list) == len(clean_df)

same_df = pd.DataFrame(same_tag_list)
diff_df = pd.DataFrame(diff_tag_list)

file_name = Path(target_file).stem

clean_df.sort_values("q_id")
same_df.sort_values("q_id")
diff_df.sort_values("q_id")

clean_df.to_csv(file_name+"_clean.csv",index=False)
same_df.to_csv(file_name+"_same.csv",index=False)
diff_df.to_csv(file_name+"_diff.csv",index=False)
                              

In [ ]:
from collections import Counter
 
print(Counter(same_df['labels']))

In [ ]:
from datasets import load_dataset
from glob import glob
import pandas as pd
from collections import Counter

csv_files = glob("*_results_*_same.csv")

split_unit = 200
test_nums = 100

dataset = load_dataset("csv", data_files={'train' : csv_files})
dataset.shuffle(42)
dataset = dataset['train'].train_test_split(test_size=test_nums)

test_dict = dataset['test']
test_dataset = pd.DataFrame(test_dict)
test_dataset.to_csv(f"test_{test_nums}_same.csv", index=False)
print(Counter(dataset['train']['labels']))
print(Counter(test_dict['labels']))

stop = False
while(not stop):
    train_dict = dataset['train'].sort('labels', reverse=True)[:split_unit]
    train_dataset = pd.DataFrame(train_dict)
    train_dataset.to_csv(f"train_{split_unit}_same_bal.csv", index=False)
    split_unit = split_unit * 2
    
    if split_unit > len(dataset['train']) - test_nums:
        stop = True
    print(Counter(train_dict["labels"]))
